In [119]:
from ur_control import transformations as tr
import quaternion as Q
from pyquaternion import Quaternion
import importlib
importlib.reload(tr)
import numpy as np
from scipy.spatial.transform import Rotation

In [120]:
# q1 = tr.random_quaternion()
# q2 = tr.random_quaternion()
q1 = [-0.05, 0.50, 0.50, 0.50]
# q1 = [0.74295004, 0.02290942, 0.62436794, 0.24013553]
# q2 = [-0.50433, 0.49562, 0.50436, 0.49561]
# q1 = np.array([0, 0, 0, 1])  # Identity quaternion
q2 = tr.rotate_quaternion_by_rpy(0, 0, np.deg2rad(10), [-0.05, 0.50, 0.50, 0.50])
dt = 0.05

In [121]:
print(q1, q2)


[-0.05, 0.5, 0.5, 0.5] [-0.09338761  0.49373956  0.54167522  0.45451948]


In [122]:
def w1(q1, q2, dt):
    delta_q = (tr.quaternion_multiply(q1, tr.quaternion_conjugate(q2)))
    delta_q_len = np.linalg.norm(delta_q[:3])
    delta_q_angle = 2*np.arctan2(delta_q_len, delta_q[3])
    return delta_q[:3] * delta_q_angle / dt


def w2(q1, q2, dt):
    q1_ = Quaternion(np.roll(q1, 1))
    q2_ = Quaternion(np.roll(q2, 1))
    delta_q = q1_*q2_.conjugate
    axis = np.linalg.norm(delta_q.vector)
    angle = 2 * np.arctan2(axis, delta_q.w)
    return delta_q.vector * angle / dt

def w3(q1, q2, dt):
    q1_ = Quaternion(np.roll(q1, 1))
    q2_ = Quaternion(np.roll(q2, 1))
    delta_q = q1_ * q2_.conjugate
    axis = delta_q.vector
    angle = 2 * np.arccos(delta_q.w)
    return axis * angle / dt

def w4(q1_, q2_, dt):
    q1 = Quaternion(np.roll(q1_, -1))
    q2 = Quaternion(np.roll(q2_, -1))
    return (2 / dt) * np.array([
        q1[0]*q2[1] - q1[1]*q2[0] - q1[2]*q2[3] + q1[3]*q2[2],
        q1[0]*q2[2] + q1[1]*q2[3] - q1[2]*q2[0] - q1[3]*q2[1],
        q1[0]*q2[3] - q1[1]*q2[2] + q1[2]*q2[1] - q1[3]*q2[0]])

def check(q1_, q2_, w, dt):
    q1 = Quaternion(np.roll(q1_, 1))
    q2 = Quaternion(np.roll(q2_, 1))
    q1.integrate(w, dt)
    dq = q2 * q1.conjugate
    angle = 2 * np.arccos(dq.w)
    return np.rad2deg(angle)
    
dt = 0.1
print("q2", q2)
print(w1(q1,q2,dt), tr.integrateUnitQuaternionDMM2(q1, w1(q1,q2,dt), dt))
print(w2(q1,q2,dt), check(q1, q2, w2(q1,q2,dt), dt))
print(w3(q1,q2,dt), check(q1, q2, w3(q1,q2,dt), dt))
print(w4(q1,q2,dt), check(q1, q2, w4(q1,q2,dt), dt))


q2 [-0.09338761  0.49373956  0.54167522  0.45451948]
[ 0.          0.         -0.11446689] [-0.04713752  0.50027798  0.49713015  0.50285347]
[ 0.00000000e+00 -1.21106545e-17 -1.14466889e-01] 60.46026713032012
[ 0.00000000e+00 -1.00375766e-16 -9.48726735e-01] 60.9127464116335
[ 0.95871317 -0.78440168  0.43142093] 61.71942372353765


In [123]:
a1 = tr.angular_velocity_from_quaternions(q1, q2, dt)
a3 = tr.quaternion_to_angular_velocity(q1, q2, dt)
a4 = tr.angular_velocities(q1, q2, dt)
print(a1, tr.integrateUnitQuaternionDMM(q1, a1, dt))
print(a1, tr.integrateUnitQuaternionDMM2(q1, a1, dt))
# print(a3, tr.integrateUnitQuaternionDMM(q1, a3, dt))
# print(a4)





[-4.16333634e-16  2.08166817e-16  1.74311485e+00] [-0.09333294  0.4937499   0.54162489  0.45457945]
[-4.16333634e-16  2.08166817e-16  1.74311485e+00] [-0.09333294  0.4937499   0.54162489  0.45457945]


In [124]:
# import numpy as np


# def quaternion_to_angular_velocity(q1, q2, dt):
#     """
#     Computes the angular velocity between two unit quaternions.

#     Args:
#         q1 (np.ndarray): A 4-element array representing the first unit quaternion.
#         q2 (np.ndarray): A 4-element array representing the second unit quaternion.
#         dt (float): The time difference between the two quaternions.

#     Returns:
#         np.ndarray: A 3-element array representing the angular velocity.
#     """
#     # Compute the quaternion difference
#     q_diff = np.quaternion(*q2) * np.quaternion(*q1).conjugate()

#     # Compute the axis-angle representation of the quaternion difference
#     axis = np.array([q_diff.x, q_diff.y, q_diff.z])
#     print(axis)
#     angle = 2.0 * np.arccos(q_diff.w)
#     print(angle)

#     # Compute the angular velocity from the axis-angle representation
#     angular_velocity = axis * angle / dt

#     return angular_velocity


# def rotate(q, w, dt):
#     q = q
#     Sq = np.array([[-q[1], -q[2], -q[3]],
#                    [q[0], -q[3], q[2]],
#                    [q[3], q[0], -q[1]],
#                    [-q[2], q[1], q[0]]])
#     return np.matmul(dt/2 * Sq, np.array(w).transpose()) + q


# # Example usage
# q1 = np.array([1, 0, 0, 0])  # Identity quaternion
# # q1 = np.roll([-0.05, 0.50, 0.50, 0.50], 1)
# q2 = tr.rotate_quaternion_by_rpy(0, 0, np.deg2rad(1), [0, 0, 0, 1])

# angular_velocity = np.array([0, 0, np.deg2rad(10)])  # 90 degree rotation per second around z-axis
# dt = 1  # Time step of 0.1 seconds

# # Integrate the angular velocity to obtain the new quaternion
q2_integrated = np.quaternion(1, *(angular_velocity * dt)) * np.quaternion(*q1)
# q2_integrated2 = rotate(q1, angular_velocity, dt)

# # Compute the actual quaternion using the angular velocity and compare to the expected quaternion
# q2_expected = np.quaternion(*np.roll(q2, 1))
# print("q2 integrated:", q2_integrated)
# print("q2 integrated2:", q2_integrated2)
# print("q2 expected:  ", q2_expected)
# print("diff", q2_integrated * q2_expected.conjugate())

# # Compute the angular velocity between q1 and q2_expected
# angular_velocity_computed = quaternion_to_angular_velocity(q1, q2_expected.components, dt)
# print("angular velocity computed:  ", angular_velocity_computed)
# print("angular velocity expected:  ", angular_velocity)


NameError: name 'angular_velocity' is not defined